In [1]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder
import pickle as pk
import pandas as pd
from tensorflow import keras
import scipy
np.random.seed(0)

In [4]:
with open("../data/pickles/data_slidingwindowFalse10_samplingrate20_df.p", 'rb') as f:
  data = pk.load(f)
dataframe = data['data']
# test_dataframe = dataframe[dataframe['pid'] == "BK7610"]
test_dataframe = dataframe[dataframe['pid'] == 'BU4707']
dataframe = dataframe[(dataframe['pid'] != 'BK7610') & (dataframe['pid']!='BU4707')]


dataframe['pid'].unique()

array(['DC6359'], dtype=object)

In [5]:
# test_dataframe['tac'].unique()

In [6]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(scipy.stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)



In [7]:

# scale_columns = ['x', 'y', 'z']
# scaler = RobustScaler()
# scaler = scaler.fit(dataframe[scale_columns])
# dataframe.loc[:, scale_columns] = scaler.transform(
#   dataframe[scale_columns].to_numpy()
# )

In [8]:
TIME_STEPS = 20
STEP = 20

X_train, y_train = create_dataset(
    dataframe[['x', 'y', 'z']],
    dataframe.tac,
    TIME_STEPS,
    STEP
)


X_test, y_test = create_dataset(
    test_dataframe[['x', 'y', 'z']],
    test_dataframe.tac,
    TIME_STEPS,
    STEP
)

/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_21149/721208264.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(scipy.stats.mode(labels)[0][0])
/var/folders/7y/kyw1v_8j0g1ckfb3g6x93q1m0000gn/T/ipykernel_21149/721208264.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or

In [9]:
X_train[0]

array([[ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038]])

In [10]:
X_test[0]

array([[-0.0015, -0.0003,  0.0196],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0069,  0.0062,  0.0167],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0091,  0.0017,  0.0164],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0001,  0.0126,  0.0168],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088,  0.0039,  0.0119]])

In [11]:
X_test.shape

(11229, 20, 3)

In [12]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [45]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['acc']
)


In [46]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 

Epoch 1/20
481/481 [==============================] - 17s 28ms/step - loss: 0.4291 - acc: 0.8883 - val_loss: 0.0760 - val_acc: 1.0000
Epoch 2/20
481/481 [==============================] - 13s 27ms/step - loss: 0.2275 - acc: 0.8825 - val_loss: 1.7419e-08 - val_acc: 1.0000
Epoch 3/20
481/481 [==============================] - 13s 27ms/step - loss: 0.3955 - acc: 0.9126 - val_loss: 0.9162 - val_acc: 0.0000e+00
Epoch 4/20
481/481 [==============================] - 11s 24ms/step - loss: 0.3324 - acc: 0.8888 - val_loss: 1.0178 - val_acc: 0.0000e+00
Epoch 5/20
481/481 [==============================] - 12s 26ms/step - loss: 0.3278 - acc: 0.8639 - val_loss: 1.7327 - val_acc: 0.0000e+00
Epoch 6/20
481/481 [==============================] - 10s 22ms/step - loss: 0.2405 - acc: 0.9201 - val_loss: 0.3571 - val_acc: 1.0000
Epoch 7/20
481/481 [==============================] - 11s 23ms/step - loss: 0.2251 - acc: 0.9127 - val_loss: 0.7641 - val_acc: 0.0064
Epoch 8/20
481/481 [==========================

In [15]:
test_dataframe[test_dataframe['tac']==1].count(), test_dataframe[test_dataframe['tac']==0].count()

(pid     63900
 x       63900
 y       63900
 z       63900
 time    63900
 tac     63900
 dtype: int64,
 pid     160700
 x       160700
 y       160700
 z       160700
 time    160700
 tac     160700
 dtype: int64)

In [16]:
X_test[0]

array([[-0.0015, -0.0003,  0.0196],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0069,  0.0062,  0.0167],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0091,  0.0017,  0.0164],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0001,  0.0126,  0.0168],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088,  0.0039,  0.0119]])

In [17]:
model.evaluate(X_test, y_test)

351/351 [==============================] - 3s 9ms/step - loss: 0.6338 - acc: 0.7155


[0.6337936520576477, 0.7154688835144043]

In [18]:
dataframe[dataframe['tac']==1].count(), dataframe[dataframe['tac']==0].count(), 

(pid     46140
 x       46140
 y       46140
 z       46140
 time    46140
 tac     46140
 dtype: int64,
 pid     264720
 x       264720
 y       264720
 z       264720
 time    264720
 tac     264720
 dtype: int64)

In [19]:
dataframe.shape

(310860, 6)

In [20]:
# test = np.squeeze(y_test)
np.where(np.squeeze(y_train))==1

False

In [22]:
# len(np.where(test == 1))

In [23]:
X_test[0]

array([[-0.0015, -0.0003,  0.0196],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0069,  0.0062,  0.0167],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0091,  0.0017,  0.0164],
       [ 0.0091,  0.0017,  0.0164],
       [-0.0015, -0.0003,  0.0196],
       [ 0.001 ,  0.0096,  0.0221],
       [ 0.0088,  0.0039,  0.0119],
       [-0.0094,  0.0147,  0.0186],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.001 ,  0.0096,  0.0221],
       [-0.0001,  0.0126,  0.0168],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088, -0.0004,  0.0177],
       [ 0.0088,  0.0039,  0.0119]])

In [24]:
X_train[0]

array([[ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0045, -0.0085, -0.0069],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038],
       [ 0.0003, -0.0001,  0.0038]])

In [ ]:
# y_train.shape

(26763, 8)

In [ ]:
# model.save("bidirectional_lstm.h5")

In [61]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=256,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='sigmoid'))

model.compile(
  loss='binary_crossentropy',
  optimizer=keras.optimizers.Adam(0.01),
  metrics=['acc']
)


history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32, 
    validation_split=0.01,
    shuffle=False) 


Epoch 1/20
481/481 [==============================] - 17s 28ms/step - loss: 0.7527 - acc: 0.9820 - val_loss: 2.3289e-04 - val_acc: 1.0000
Epoch 2/20
481/481 [==============================] - 10s 21ms/step - loss: 0.8996 - acc: 0.9541 - val_loss: 0.0026 - val_acc: 1.0000
Epoch 3/20
481/481 [==============================] - 13s 28ms/step - loss: 0.4427 - acc: 0.8460 - val_loss: 2.3683 - val_acc: 0.0000e+00
Epoch 4/20
481/481 [==============================] - 11s 23ms/step - loss: 0.4126 - acc: 0.8603 - val_loss: 2.4487 - val_acc: 0.0000e+00
Epoch 5/20
481/481 [==============================] - 10s 21ms/step - loss: 0.4163 - acc: 0.8603 - val_loss: 2.4465 - val_acc: 0.0000e+00
Epoch 6/20
481/481 [==============================] - 12s 26ms/step - loss: 0.4167 - acc: 0.8603 - val_loss: 2.4391 - val_acc: 0.0000e+00
Epoch 7/20
481/481 [==============================] - 10s 21ms/step - loss: 0.4167 - acc: 0.8603 - val_loss: 2.4339 - val_acc: 0.0000e+00
Epoch 8/20
481/481 [==================

In [62]:
model.evaluate(X_test, y_test)

351/351 [==============================] - 3s 9ms/step - loss: 0.7566 - acc: 0.7156


[0.7565633058547974, 0.7155579328536987]

In [ ]:
X_test.shape

(30725, 200, 3)

In [ ]:
X_test.shape

(30725, 200, 3)

In [ ]:
test_dataframe.head()

,pid,x,y,z,time,tac
47150,BK7610,0.1016,-0.0145,-0.0042,1493735870,0
47144,BK7610,0.1076,0.0032,0.0276,1493735870,0
47148,BK7610,0.1085,-0.0282,-0.0035,1493735870,0
47160,BK7610,0.0918,-0.0076,-0.0020,1493735870,0
47140,BK7610,0.1443,-0.0474,-0.0447,1493735870,0


In [25]:
# enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

# enc = enc.fit(y_train)

# y_train = enc.transform(y_train)
# enc.transform(y_test)

## MLP Classifier

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf = MLPClassifier(solver='adam', shuffle=True, random_state=1)

In [28]:
X_train.shape

(15542, 20, 3)

In [30]:
clf.fit(np.reshape(X_train, (15542, 60)), y_train)
clf.get_params()

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [31]:
X_test.shape

(11229, 20, 3)

In [32]:
print('Accuracy ', accuracy_score(y_test, clf.predict(np.reshape(X_test, (11229, 60)))))

Accuracy  0.7000623385875857


## CNN 

In [ ]:
batch_size = 256
epochs = 1000


flatten = keras.layers.Flatten()
conv_layer1 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
conv_layer2 = keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
dropout = keras.layers.Dropout(0.5)
max_pooling = keras.layers.MaxPool1D(pool_size=2)
fc_layer = keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = keras.layers.Dense(units=1, activation = 'sigmoid')
base_model = keras.Sequential([
                                  conv_layer1,  
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

base_model.compile(loss='binary_crossenttropy', 
                   optimizer='adam',
                   metrics=['acc']) 